<a href="https://colab.research.google.com/github/jnsbrdbr/hopefully_last_thesis/blob/main/992_1000_PCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import jaccard_score
from scipy.spatial.distance import pdist, squareform

from sklearn.decomposition import PCA
import copy

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **importing data**

In [3]:
import tables
tables.file._open_files.close_all()

In [4]:
store = pd.HDFStore('/content/drive/MyDrive/1000/Tdata.h5')

adj=store['adj']
store.close()

In [5]:
with pd.HDFStore('/content/drive/MyDrive/1000/TCFdf.h5',  mode='r') as newstore:
    CF = newstore.select('CFdf')

In [6]:
store = pd.HDFStore('/content/drive/MyDrive/1000/TTFdf.h5')
TF=store['TFdf']
store.close()

In [7]:
with pd.HDFStore('/content/drive/MyDrive/1000/TjaccardsimilaritydfR.h5',  mode='r') as newstore:
    jaccardsimilarity = newstore.select('jaccardsimilaritydfR')

In [8]:
store = pd.HDFStore('/content/drive/MyDrive/1000/TcosinesimilaritydfR.h5')
cosine=store['cosine_similaritydfR']

In [9]:
store = pd.HDFStore('/content/drive/MyDrive/1000/TL1normdf.h5')
L1_normdf=store['L1_normdf']
store.close()

In [10]:
store = pd.HDFStore('/content/drive/MyDrive/1000/Tsum.h5')
wsumdf=store['wsumdf']
store.close()

In [11]:
with pd.HDFStore('/content/drive/MyDrive/1000/Tmul.h5',  mode='r') as newstore:
    wmuldf = newstore.select('wmuldf')

In [12]:
with pd.HDFStore('/content/drive/MyDrive/1000/TLdata.h5',  mode='r') as newstore:
    adjl = newstore.select('adjl')

# **implementation of PCA**

In [13]:
pca = PCA(n_components=10)
CF_pca = pca.fit_transform(CF)
CF_pca_df=pd.DataFrame(CF_pca)

In [14]:
pca = PCA(n_components=10)
TF_pca = pca.fit_transform(TF)
TF_pca_df=pd.DataFrame(TF_pca)

In [25]:
pca = PCA(n_components=10)
jaccard_similarity_pca = pca.fit_transform(jaccardsimilarity)
jaccard_similarity_pca_df=pd.DataFrame(jaccard_similarity_pca)

In [16]:
pca = PCA(n_components=10)
cosine_similarity_pca = pca.fit_transform(cosine)
cosine_similarity_pca_df=pd.DataFrame(cosine_similarity_pca)

In [17]:
pca = PCA(n_components=10)
L1_normdf_similarity_pca = pca.fit_transform(L1_normdf)
L1_normdf_similarity_pca_df=pd.DataFrame(L1_normdf_similarity_pca)

In [18]:
pca = PCA(n_components=10)
wsumdf_similarity_pca = pca.fit_transform(wsumdf)
wsumdf_similarity_pca_df=pd.DataFrame(wsumdf_similarity_pca)

In [19]:
pca = PCA(n_components=10)
wmuldf_similarity_pca = pca.fit_transform(wmuldf)
wmuldf_similarity_pca_df=pd.DataFrame(wmuldf_similarity_pca)

In [21]:
y=adjl.label
y_arr=np.array(y)
y_df=pd.DataFrame(y_arr)

In [26]:
#concat
frames = [CF_pca_df, TF_pca_df, jaccard_similarity_pca_df,cosine_similarity_pca_df,
          L1_normdf_similarity_pca_df,wsumdf_similarity_pca_df,wmuldf_similarity_pca_df,y_df]
result = pd.concat(frames,axis=1)

In [27]:
result

,0,1,2,3,4,5,6,7,8,9,...,1,2,3,4,5,6,7,8,9,0
0,-1.715972,-3.099936,-0.041249,4.793804e-14,1.533198e-14,0.077295,1.691428,3.341114,1.720130e-13,-4.141169e-13,...,7.286881e-03,9.133387e-03,6.177951e-03,-1.766329e-02,-1.546903e-03,-2.129364e-03,-2.060071e-02,-1.211687e-02,6.798811e-03,1
1,-1.715972,-3.099936,-0.041249,-1.573467e-14,3.463010e-14,0.077295,1.691428,3.341114,-2.869672e-14,-1.343755e-13,...,-7.592618e-03,1.203724e-02,1.119539e-03,4.892798e-03,-1.444263e-02,-2.126158e-02,-3.820006e-02,-4.360685e-03,1.891001e-02,1
2,-1.715972,-3.099936,-0.041249,-1.049169e-13,-2.150311e-13,0.077295,1.691428,3.341114,4.544143e-15,-9.468718e-14,...,-4.577336e-04,1.930910e-02,-4.886899e-05,-3.417111e-03,2.228885e-02,3.994949e-03,-6.179284e-03,-4.344655e-03,1.311346e-02,1
3,-1.715972,-3.099936,-0.041249,-5.202873e-14,-2.099344e-13,0.077295,1.691428,3.341114,-5.773652e-15,3.295284e-14,...,-2.039012e-02,1.150549e-02,-1.058895e-02,-5.315466e-03,-7.836788e-03,-2.482258e-03,1.817531e-03,-1.231243e-03,-3.614036e-03,1
4,-1.715972,-3.099936,-0.041249,1.104825e-14,-6.527418e-14,0.077295,1.691428,3.341114,-4.720397e-15,5.116521e-14,...,6.126758e-15,2.462819e-15,8.188884e-15,7.798825e-15,3.682839e-15,8.178555e-15,-1.445560e-14,-1.809574e-13,1.565862e-15,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,-2.114893,-4.572474,-0.754679,5.571174e-13,-3.262912e-13,8.797502,-0.661164,-0.063010,-2.076237e-02,5.177774e-02,...,-1.937338e-02,2.294438e-02,6.965943e-03,6.905981e-03,2.042971e-02,8.835102e-04,-2.721377e-02,6.363850e-03,1.567893e-02,1
996,-2.114893,-4.572474,-0.754679,5.572066e-13,-3.257378e-13,8.797502,-0.661164,-0.063010,3.633826e-03,-1.461719e-02,...,-2.162240e-02,1.731516e-02,-1.492769e-02,2.406943e-02,-3.219238e-03,1.239865e-02,4.898372e-02,-5.092807e-03,-1.611609e-02,1
997,-2.114893,-4.572474,-0.754679,5.571203e-13,-3.262280e-13,8.797502,-0.661164,-0.063010,3.482977e-02,2.536692e-02,...,7.172889e-05,-1.345066e-02,1.013830e-02,2.224863e-04,9.640867e-03,1.964005e-02,1.425644e-02,-3.650507e-03,1.049643e-02,1
998,-2.114893,-4.572474,-0.754679,5.573195e-13,-3.257510e-13,8.797502,-0.661164,-0.063010,9.614899e-03,1.143072e-02,...,-1.167698e-02,3.470006e-02,-2.897900e-03,-7.920176e-03,-1.667178e-02,1.509620e-02,1.630737e-02,-2.939188e-03,-2.392343e-02,1


In [39]:
last_column = result.iloc[: , -1]

In [40]:
with pd.HDFStore('/content/drive/MyDrive/last/last_result_1000',  mode='w') as store:
    store.append('result', result,format='table')